In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from flask import Flask, request, jsonify
from sklearn.preprocessing import StandardScaler

# Load & preprocess biomechanical data
def load_motion_tracking_data():
    try:
        # Load dataset
        data = pd.read_csv("Horse_Biomechanics.csv")

        # Print available columns for debugging
        print("Columns in dataset:", data.columns.tolist())

        # Define feature columns
        features = ["Stride_Length (m)", "Acceleration (m/s^2)", "Speed (km/h)"]

        # Ensure dataset has required columns
        for col in features:
            if col not in data.columns:
                raise KeyError(f"Missing column in dataset: {col}")

        # Compute `Performance_Score` if missing
        if "Performance_Score" not in data.columns:
            print("⚠️ Performance_Score column not found. Computing dynamically...")
            data["Performance_Score"] = (
                0.4 * data["Stride_Length (m)"] + 
                0.3 * data["Acceleration (m/s^2)"] + 
                0.3 * data["Speed (km/h)"]
            )

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(data[features])
        y = data["Performance_Score"].values  # Target variable

        return X_scaled, y, scaler

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

# Load data
result = load_motion_tracking_data()
if result is None:
    raise RuntimeError("Dataset processing failed. Please check your CSV file.")
else:
    X_train, y_train, scaler = result

# Define LSTM Model
def build_lstm_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, activation='relu', return_sequences=True, input_shape=input_shape),
        tf.keras.layers.LSTM(50, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Train Model
input_shape = (X_train.shape[1], 1)
model = build_lstm_model(input_shape)

# Reshape input for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Save trained model
model.save("EquiSync_Model.h5")

# Initialize Flask API
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        input_df = pd.DataFrame([data])

        # Define required features
        required_features = ["Stride_Length (m)", "Acceleration (m/s^2)", "Speed (km/h)"]

        # Ensure all required features exist
        missing_features = [f for f in required_features if f not in input_df.columns]
        if missing_features:
            return jsonify({'error': f"Missing features: {missing_features}"}), 400

        # Scale input data
        input_scaled = scaler.transform(input_df[required_features])
        input_reshaped = np.expand_dims(input_scaled, axis=-1)  # Reshape for LSTM

        # Make prediction
        prediction = model.predict(input_reshaped)

        return jsonify({'Performance_Score': float(prediction[0][0])})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)  # Use Flask's built-in server


Columns in dataset: ['Horse_ID', 'Stride_Length (m)', 'Gait_Pattern', 'Acceleration (m/s^2)', 'Speed (km/h)', 'Timestamp']
⚠️ Performance_Score column not found. Computing dynamically...


C:\Users\pc\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 230.6857 - mae: 14.9552 - val_loss: 220.6849 - val_mae: 14.6259
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 222.6598 - mae: 14.6940 - val_loss: 179.4254 - val_mae: 13.1419
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 137.0571 - mae: 11.0982 - val_loss: 46.0096 - val_mae: 5.5442
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 33.7408 - mae: 4.8308 - val_loss: 24.4956 - val_mae: 4.0846
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 21.7826 - mae: 3.8821 - val_loss: 14.6859 - val_mae: 3.1232
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 13.1302 - mae: 2.9780 - val_loss: 10.2017 - val_mae: 2.5762
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.9920 - mae: 2.4388 - val_loss: 7.9883 - val_mae: 2.2910
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 7.6818 - mae: 2.2713 - val_loss: 6.6807 - val_mae: 2.1120
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.44:5000
INFO:werkzeug:Press CTRL+C to quit


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load datasets
biomechanics_df = pd.read_csv("Horse_Biomechanics.csv")
vitals_df = pd.read_csv("Horse_VitalSigns.csv")
performance_df = pd.read_csv("Race_Performance.csv")

# Merge datasets on Horse_ID and Timestamp
merged_df = biomechanics_df.merge(vitals_df, on=["Horse_ID", "Timestamp"]).merge(performance_df, on="Horse_ID")

# Selecting relevant features
features = ["Stride_Length (m)", "Acceleration (m/s^2)", "Speed (km/h)", "Heart_Rate (bpm)", "Oxygen_Level (%)"]
target = "Performance_Score"

# Data preprocessing
scaler = MinMaxScaler()
X = scaler.fit_transform(merged_df[features])
y = merged_df[target].values

# Reshape for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16)

# Save model
model.save("equisync_model.h5")


C:\Users\pc\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 54.5743 - val_loss: 32.7132
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 18.9410 - val_loss: 3.5072
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.3367 - val_loss: 3.1437
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.8551 - val_loss: 2.8400
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.1555 - val_loss: 2.6132
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1348 - val_loss: 2.4415
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.7357 - val_loss: 2.3397
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.6172 - val_loss: 2.2276
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4290 - val_loss: 2.1395
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6039 - val_loss: 2.2233
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.5032 - val_loss: 2.0403
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.3454 - va

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from flask import Flask, request, jsonify
from sklearn.preprocessing import MinMaxScaler

# Define custom objects if needed
custom_objects = {"mse": tf.keras.losses.MeanSquaredError()}

# Load trained model with custom objects and without compiling
model = tf.keras.models.load_model("equisync_model.h5", custom_objects=custom_objects, compile=False)

# Initialize Flask app
app = Flask(__name__)

# Define a scaler to match training preprocessing
scaler = MinMaxScaler()

def preprocess_input(data):
    features = ["Stride_Length", "Acceleration", "Speed", "Heart_Rate", "Oxygen_Level"]
    df = pd.DataFrame([data], columns=features)
    df_scaled = scaler.fit_transform(df)  # Use transform instead of fit_transform for consistency
    return df_scaled.reshape((1, 1, len(features)))

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()
        processed_data = preprocess_input(data)
        prediction = model.predict(processed_data)
        return jsonify({"Performance_Score": float(prediction[0][0])})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\pc\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
